In [19]:
import sys
import yaml
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm.auto import tqdm

# 1. Setup path to import our custom modules from the src folder
sys.path.append('..')
from src.data_loader import MicrogridDataLoader
from src.model import MicrogridLSTM

# 2. Load Configuration
with open("../config.yaml", "r") as f:
    config = yaml.safe_load(f)
    
# Fix the raw_path since we are running this inside the 'notebooks' folder
config['data']['raw_path'] = "../" + config['data']['raw_path']

# 3. Load and Preprocess Data
print("⏳ Loading and preprocessing data...")
loader = MicrogridDataLoader(config)
data_scaled = loader.load_and_preprocess()
X, y = loader.create_sequences(data_scaled)

# 4. Train/Test Split (Driven by config.yaml!)
test_size = config['data']['test_size']
split_idx = int(len(X) * (1 - test_size))

X_train_raw, X_test_raw = X[:split_idx], X[split_idx:]
y_train_raw, y_test_raw = y[:split_idx], y[split_idx:]

# Convert to PyTorch Tensors
X_train = torch.tensor(X_train_raw, dtype=torch.float32)
y_train = torch.tensor(y_train_raw, dtype=torch.float32).reshape(-1, 1)
X_test = torch.tensor(X_test_raw, dtype=torch.float32)
y_test = torch.tensor(y_test_raw, dtype=torch.float32).reshape(-1, 1)

print(f"✅ Data ready! Training on {len(X_train)} samples, Testing on {len(X_test)} samples.")

# 5. Initialize the Model (Driven by config.yaml!)
input_features = X_train.shape[2]
hidden_size = config['model']['hidden_size']
num_layers = config['model']['num_layers']
output_window = config['model']['output_window']

model = MicrogridLSTM(input_features, hidden_size, num_layers, output_window)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=config['model']['learning_rate'])

# 6. Training Loop
epochs = config['model']['epochs']
batch_size = config['model']['batch_size']

print(f"🚀 Starting Training: {epochs} Epochs, {input_features} Features...")

try:
    for epoch in range(epochs):
        model.train()
        
        # Batch progress bar
        pbar = tqdm(range(0, len(X_train), batch_size), desc=f"Epoch {epoch+1}/{epochs}", leave=False)
        
        for i in pbar:
            X_batch = X_train[i : i + batch_size]
            y_batch = y_train[i : i + batch_size]
            
            # Forward pass
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # Update progress bar text
            pbar.set_description(f"Loss: {loss.item():.4f}")
            
    print("✨ Training Complete!")

except KeyboardInterrupt:
    print("\n🛑 Training stopped by user. Model weights are preserved up to this point.")

ImportError: cannot import name 'MicrogridDataLoader' from 'src.data_loader' (c:\Portfolio\renewable-forecasting-dl\notebooks\..\src\data_loader.py)